<a href="https://colab.research.google.com/github/componavt/neural_synset/blob/master/%20%20%20%20src/dataset/wikt_labels_loading_a_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading a custom dataset

Source code: [Loading a custom dataset](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/videos/load_custom_dataset.ipynb#scrollTo=D2ekPOyykZDq), [video](https://www.youtube.com/watch?v=HyQgpJTkRdE).

Video: [The pipeline function](https://www.youtube.com/watch?v=tiZFewofSLM).

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
! pip install datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


In [2]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

--2024-03-21 08:23:51--  https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/componavt/neural_synset/master/data/label_meaning.csv [following]
--2024-03-21 08:23:51--  https://raw.githubusercontent.com/componavt/neural_synset/master/data/label_meaning.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1227 (1.2K) [text/plain]
Saving to: ‘label_meaning.csv’

label_meaning.csv   100%[===================>]   1.20K  --.-KB/s    in 0s      

2024-03-21 08:23:52 (43.8 MB/s) - ‘label_meaning.csv’ saved [1227/1227]



In [3]:
cat label_meaning.csv

"word"|"meaning"|"книжн."|"ирон."|"религ."|"груб."
подвизаться|осуществлять деятельность, работать, действовать в какой-нибудь области|1|1|0|0
подвизаться|совершать подвиг в чём-либо, часто о ежедневном борении|0|0|1|0
заткнуться|то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть|0|0|0|1
пустобрёх|тот, кто говорит много пустого и несерьёзного; болтун|0|0|0|1
излаять|сильно изругать|0|0|0|1
бизнес-дама|о предпринимательнице|0|1|0|0
агнец божий|кроткий, робкий, безобидный человек|0|1|0|0
всезнайка|человек, который считает себя знающим всё|0|1|0|0
галантерейный|относящийся к галантерее|0|0|0|0
галантерейный|чрезмерно любезный, вежливый до слащавости|0|1|0|0
дитятя|дитя, ребёнок, чадо|0|1|0|0


In [14]:
from datasets import load_dataset

#ds = load_dataset("csv", data_files="label_meaning.csv", sep="|")
ds = load_dataset("csv", data_files="label_meaning.csv", sep="|", split='train')
#ds["train"]
ds

Dataset({
    features: ['word', 'meaning', 'книжн.', 'ирон.', 'религ.', 'груб.'],
    num_rows: 11
})

In [54]:
# 80% train, 20% test + validation
datushka = ds.train_test_split(test_size=0.2, shuffle=True)
datushka
#datushka["train"]
#datushka["test"]

DatasetDict({
    train: Dataset({
        features: ['word', 'meaning', 'книжн.', 'ирон.', 'религ.', 'груб.'],
        num_rows: 8
    })
    test: Dataset({
        features: ['word', 'meaning', 'книжн.', 'ирон.', 'религ.', 'груб.'],
        num_rows: 3
    })
})

In [63]:
#print(datushka["train"][0])
print(len(list(datushka["train"])))

8


# Pipeline: zero shot classification with labels: education, business and politics

When more than one label is passed, we assume that there is only one true label and that the others are false so that the output probabilities add up to 1. This can be changed by passing `multi_class=True`:
nlp(sequence_to_classify, candidate_labels, multi_class=True)

Source: huggingface/transformers/[Zero shot classification pipeline #5760 ](https://github.com/huggingface/transformers/pull/5760).

In [65]:
from transformers import pipeline
model_id = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
pipe = pipeline("zero-shot-classification", model=model_id)

In [69]:
def meaning_iterator():
    for i in range(0, len(datushka["train"]), 1):
        yield datushka["train"][i]["meaning"]

#print(len(datushka["train"]))
#datushka["train"][0]["meaning"]
#nlp(datushka["train"][0]["meaning"], ["positive", "negative"], multi_label=True)
###pipe(meaning_iterator(), ["positive", "negative"], multi_label=True)

8


In [72]:
#sequence_to_classify = "тот, кто говорит много пустого и несерьёзного; болтун"
candidate_labels = ["книжн.", "ирон.", "религ.", "груб."]
#pipe(sequence_to_classify, candidate_labels, multi_label=True)

#pipe(meaning_iterator(), candidate_labels, multi_label=True)
pipe(datushka["train"][0]["meaning"], candidate_labels, multi_label=True)

{'sequence': 'тот, кто говорит много пустого и несерьёзного; болтун',
 'labels': ['груб.', 'ирон.', 'книжн.', 'религ.'],
 'scores': [0.9954854249954224,
  0.9225433468818665,
  0.21135631203651428,
  0.18404583632946014]}